In [9]:
import tensorflow as tf
import numpy as np
import matplotlib as plt
import konlpy

In [13]:
cleaned_corpus = list(set(raw))

filtered_corpus = [s for s in cleaned_corpus if (len(s) < max_len) & (len(s) >= min_len)]

In [15]:
import sentencepiece as spm
import os
temp_file = os.getenv('HOME')+'/aiffel/sp_tokenizer/data/korean-english-park.train.ko.temp'

vocab_size = 8000

with open(temp_file, 'w') as f:
    for row in filtered_corpus:
        f.write(str(row) + '\n')
        
spm.SentencePieceTrainer.Train(
'--input={} --model_prefix=korean_spm --vocab_size={}'.format(temp_file, vocab_size))

!ls -l korean_spm*

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=/aiffel/aiffel/sp_tokenizer/data/korean-english-park.train.ko.temp --model_prefix=korean_spm --vocab_size=8000
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /aiffel/aiffel/sp_tokenizer/data/korean-english-park.train.ko.temp
  input_format: 
  model_prefix: korean_spm
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id

-rw-r--r-- 1 root root 376725 Jan 15 14:13 korean_spm.model
-rw-r--r-- 1 root root 146094 Jan 15 14:13 korean_spm.vocab


In [16]:
s = spm.SentencePieceProcessor()
s.Load('korean_spm.model')

tokensIDs = s.EncodeAsIds("아버지가방에들어가신다.")
print(tokensIDs)

print(s.SampleEncodeAsPieces("아버지가방에들어가신다", 1, 0.0))

print(s.DecodeIds(tokensIDs))

[1255, 11, 304, 7, 3606, 11, 285, 38, 3]
['▁아버지', '가', '방', '에', '들어', '가', '신', '다']
아버지가방에들어가신다.


In [19]:
def sp_tokenize(s, corpus):
    
    tensor = []
    
    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))
        
    with open("./korean_spm.vocab", 'r') as f:
        vocab = f.readlines()
        
    word_index = {}
    index_word = {}
    
    for idx, line in enumerate(vocab):
        word = line.split('\t')[0]
        
        word_index.update({idx:word})
        index_word.update({word:idx})
        
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    return tensor, word_index, index_word

In [20]:
my_corpus = ["나는 밥을 먹었습니다.", "그러나 여전히 ㅠㅠ 배가 고픕니다..."]
tensor, word_index, index_word = sp_tokenize(s, my_corpus)
print(tensor)

[[1957 5607    5    4 7975 2012    3    0    0    0    0    0    0    0]
 [ 108 1658  101    4    0  470   11    4   14    0 2002    3    3    3]]
